In [ ]:
from bottle import Bottle, route, run, template, get, post, debug, static_file, request, redirect, response
import time
import random
import string
import logging
import logging.handlers
import sqlite3
import requests
import json
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
from tqdm import tqdm_notebook as tqdm # Progress bar
# tqdm().pandas()
from IPython.display import display, HTML

log = logging.getLogger('bottle')
log.setLevel('INFO')
h = logging.handlers.TimedRotatingFileHandler(
    'logs/nlog', when='midnight', backupCount=9999)
f = logging.Formatter('%(asctime)s %(levelname)-8s %(message)s')
h.setFormatter(f)
log.addHandler(h)

secretKey = "vcxv&TTFS98v7xcfvhjghd6DFOUFYWE&FY"
loginName = ''
password = ''



@route('/static/<path:path>', name='static')
def static(path):
    return static_file(path, root='./static')


def CheckIfCorrectCredentials():
    loginName = request.get_cookie("user", secret=secretKey)
    password = request.get_cookie("password", secret=secretKey)
    print(loginName, password)
    randStr = request.get_cookie("randStr", secret=secretKey)
    log.info(str(loginName) + ' ' + request.method + ' ' +
             request.url + ' ' + request.environ.get('REMOTE_ADDR'))
    
    conn = sqlite3.connect('cinema_AS_SS.db')
    c = conn.cursor()

    params=[loginName, password]
    c.execute("Select * from Customers where login==? and password==?", params)
    print("\nPresent all data in student table:")
    
    #Get all results,assign them to the list,fecthall() returns empty list if no results
    listOfResults=c.fetchall()
    print(c.fetchall())
    conn.commit()
    conn.close()
    
    if (len(listOfResults)==1):
        return loginName
    return redirect('/login')

#stare
# def checkAuth():
#     loginName = request.get_cookie("user", secret=secretKey)
#     randStr = request.get_cookie("randStr", secret=secretKey)
#     log.info(str(loginName) + ' ' + request.method + ' ' +
#              request.url + ' ' + request.environ.get('REMOTE_ADDR'))
#     if (loginName in users) and (users[loginName].get("randStr", "") == randStr) and (users[loginName]["loggedIn"] == True) and (time.time() - users[loginName]["lastSeen"] < 3600):
#         users[loginName]["lastSeen"] = time.time()
#         return loginName
#     return redirect('/login')


@route('/login')
@route('/login/')
@route('/login', method='POST')
def login():
    loginName = request.forms.get('login_name', default=False)
    password = request.forms.get('password', default=False)
    print(loginName, password)
    
    randStr = ''.join(random.choice(
        string.ascii_uppercase + string.digits) for _ in range(18))
    log.info(str(loginName) + ' ' + request.method + ' ' +
             request.url + ' ' + request.environ.get('REMOTE_ADDR'))
    
    conn = sqlite3.connect('cinema_AS_SS.db')
    c = conn.cursor()

    params=[loginName, password]
    c.execute("Select * from Customers where login==? and password==?", params)
    print("\nPresent all data in student table:")
    
    #Get all results,assign them to the list,fecthall() returns empty list if no results
    listOfResults=c.fetchall()
    print(c.fetchall())
    conn.commit()
    conn.close()
    
    if (len(listOfResults)==1):
        response.set_cookie("user", loginName, secret=secretKey)
        response.set_cookie("password", password, secret=secretKey)
        response.set_cookie("randStr", randStr, secret=secretKey)

        redirect('/login_suc')
        return True
    
    else:
        template('login', title='Log In')
    return template('login', title='Log In')


@route('/login_suc')
@route('/login_suc/')
def login_suc():
    
    loginName = CheckIfCorrectCredentials()
    if (loginName == "arttt1" or loginName == "szymm2"):
        return template('admin_login_suc', title='Success! You are logged in!', loginName = request.get_cookie("user", secret=secretKey))
    
    return template('login_suc', title='Success! You are logged in!', loginName = request.get_cookie("user", secret=secretKey))


@route('/register')
@route('/register/')
@route('/register', method='GET')
def register():
    
    conn = sqlite3.connect('cinema_AS_SS.db')
    c = conn.cursor()
    
    if request.GET.save:
        
        new_login = request.query.login
        new_password = request.query.password
        new_name = request.query.name
        new_surname = request.query.surname
        new_mobile = request.query.mobile
        new_email = request.query.email
        new_address = request.query.address
        
        insert_new_customers = (
            """INSERT INTO Customers(login, password, name, surname, mobile, email, address)  VALUES(?, ?, ?, ?, ?, ?, ?)"""
        )
        
        new_customer = (new_login, new_password, new_name, new_surname, new_mobile, new_email, new_address)
        
        c.execute(insert_new_customers, new_customer)
               
#         new_row = c.execute("SELECT * FROM Customers ORDER BY login DESC LIMIT 1");
        
        conn.commit()
        conn.close()
        
        return template('registered.tpl', title = "Welcome")
    else:
        if (loginName == "arttt1" or loginName == "szymm2"):
            return template('admin_register.tpl', title = "Register")
        
        return template('register.tpl', title = "Register")

#stare
#     loginName = request.forms.getunicode('login_name', default=False)
#     password = request.forms.getunicode('password', default=False)
#     randStr = ''.join(random.choice(
#         string.ascii_uppercase + string.digits) for _ in range(18))
#     if (loginName in users) and users[loginName]["password"] == password:
#         response.set_cookie("user", loginName, secret=secretKey)
#         response.set_cookie("randStr", randStr, secret=secretKey)
#         users[loginName]["loggedIn"] = True
#         users[loginName]["randStr"] = randStr
#         users[loginName]["lastSeen"] = time.time()
        
#         redirect('/index')
#         return True
#     else:
#         return template('login', title='Log In')
#     return template('login', title='Log In')


@route('/')
@route('/index')
@route('/index/')
def glowna():
    
    loginName = request.get_cookie("user", secret=secretKey)
    
    if (loginName == "arttt1" or loginName == "szymm2"):
        return template('admin_index', title = "Home page")    
    
    return template('index', title='Home page')


@route('/buy')
@route('/buy/')
def buy_ticket():

    loginName = CheckIfCorrectCredentials()
    
    conn = sqlite3.connect('cinema_AS_SS.db')        
    c = conn.cursor()
    c.execute("SELECT a.id_movie, a.movie_name, a.date, a.time, b.room \
               FROM Movie as a \
               LEFT JOIN Rooms as b \
               on a.id_room = b.id_room")
    listOut=c.fetchall()
    conn.commit()
    # close the connection and free all resources
    conn.close()    
    #./views/allStudents.tpl file contains html code which creates tables based on data in res variable 
    #Remark: allStudents link was added to navigation bar in ./views/base.tpl 
    
    #loginName = request.get_cookie("user", secret=secretKey)
       
    return template('buy_ticket', title='Buy ticket', res=listOut)

# nieudana proba gdzie proborwalem tworzyc matrix jako sale

# @route('/buy/<id_room:path>')
# @route('/buy/<id_room:path>/')
# def buy_ticket_movie(id_room):

#     conn = sqlite3.connect('cinema_AS_SS.db')        
#     c = conn.cursor()
#     query = ("""SELECT Movie.id_movie, Rooms.id_room, Rooms.room, Tickets.row, Tickets.seat_number
#               FROM Tickets
#               LEFT JOIN Movie
#               ON Tickets.id_movie = Movie.id_movie
#               LEFT JOIN Rooms
#               ON Movie.id_room = Rooms.id_room
#               WHERE Movie.id_room == ?""") 
    
#     c.execute(query, (id_room, ))

#     listOut=c.fetchall()
#     conn.commit()
#     # close the connection and free all resources
#     conn.close()    
#     #./views/allStudents.tpl file contains html code which creates tables based on data in res variable 
#     #Remark: allStudents link was added to navigation bar in ./views/base.tpl 
#     #loginName = request.get_cookie("user", secret=secretKey)
# #     if room == 1:       
# #         return template('movie_rooms_1', title='Rooms', res=listOut)
#     return template('movie_rooms_2', title='Rooms', res=listOut)

@route('/buy/<id_movie:path>')
@route('/buy/<id_movie:path>/')
def do_upload(id_movie):
    
    loginName = CheckIfCorrectCredentials()
    
    conn = sqlite3.connect('cinema_AS_SS.db')
    c = conn.cursor()
    c.execute("SELECT a.id_movie, b.room \
              FROM Movie as a \
              JOIN Rooms as b \
              ON a.id_room == b.id_room \
              WHERE a.id_movie == ?", (id_movie, ))
    listOut=c.fetchall()
    conn.commit()
    conn.close()
    
    return template('movie_rooms', title = 'Rooms', res=listOut, login = loginName)


@route('/select_1/<id_movie:path>/<login:path>')
@route('/select_1/<id_movie:path>/<login:path>/')
@route('/select_1/<id_movie:path>/<login:path>', method='GET')
def register(id_movie, login):
    
    login = CheckIfCorrectCredentials()
    id_ticket = str(random.randint(0, 999))
    id_payment = random.randint(0, 999)
    conn = sqlite3.connect('cinema_AS_SS.db')
    c = conn.cursor()
    
    room = 0
    if id_movie == "10":
        room = 1
    elif id_movie == "11":
        room = 2
    else:
        room = 0
    
    if request.GET.save:
        
        new_row = request.query.row
        new_seat_number = request.query.seat_number
        new_surname = request.query.surname
        new_payment_method = request.query.payment_method
        
        insert_new_ticket = (
            """INSERT INTO Tickets(id_ticket, id_payment, login, id_movie, type, price, room, row, seat_number, payment_method)  
            VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""
        )
        
        new_ticket = (id_ticket, id_payment, login, id_movie, "Normal", 25.50, room, new_row, new_seat_number, new_payment_method) 
        
        c.execute(insert_new_ticket, new_ticket)
               
        
        conn.commit()
        conn.close()
        
        return redirect('/my_reservation')
    
    return template('select_place.tpl', title = "Select Place", id_movie = id_movie, login = login)

@route('/select_2/<id_movie:path>/<login:path>')
@route('/select_2/<id_movie:path>/<login:path>/')
@route('/select_2/<id_movie:path>/<login:path>', method='GET')
def register(id_movie, login):
    
    login = CheckIfCorrectCredentials()
    id_ticket = str(random.randint(0, 999))
    id_payment = random.randint(0, 999)
    conn = sqlite3.connect('cinema_AS_SS.db')
    c = conn.cursor()
    
    room = 0
    if id_movie == "10":
        room = 1
    elif id_movie == "11":
        room = 2
    else:
        room = 0
    
    if request.GET.save:
        
        new_row = request.query.row
        new_seat_number = request.query.seat_number
        new_surname = request.query.surname
        new_payment_method = request.query.payment_method
        
        insert_new_ticket = (
            """INSERT INTO Tickets(id_ticket, id_payment, login, id_movie, type, price, room, row, seat_number, payment_method)  
            VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""
        )
        
        new_ticket = (id_ticket, id_payment, login, id_movie, "Reduced", 12.00, room, new_row, new_seat_number, new_payment_method) 
        
        c.execute(insert_new_ticket, new_ticket)
               
        
        conn.commit()
        conn.close()
        
        return redirect('/my_reservation')
    
    return template('select_place.tpl', title = "Select Place", id_movie = id_movie, login = login)

@route('/movies')
@route('/movies/')
def movies():

    conn = sqlite3.connect('cinema_AS_SS.db')        
    c = conn.cursor()
    c.execute("SELECT movie_name, date, time FROM Movie")
    listOut=c.fetchall()
    conn.commit()
    # close the connection and free all resources
    conn.close()    
    #./views/allStudents.tpl file contains html code which creates tables based on data in res variable 
    #Remark: allStudents link was added to navigation bar in ./views/base.tpl 
    
    #loginName = request.get_cookie("user", secret=secretKey)
       
    return template('movies', title='Movies', res=listOut)

@route('/admin_movies')
@route('/admin_movies/')
def movies():
    
    loginName = CheckIfCorrectCredentials()

    if (loginName == "arttt1" or loginName == "szymm2"):
    
        conn = sqlite3.connect('cinema_AS_SS.db')        
        c = conn.cursor()
        c.execute("SELECT movie_name, date, time FROM Movie")
        listOut=c.fetchall()
        conn.commit()
    # close the connection and free all resources
        conn.close()    
    #./views/allStudents.tpl file contains html code which creates tables based on data in res variable 
    #Remark: allStudents link was added to navigation bar in ./views/base.tpl 
    
    #loginName = request.get_cookie("user", secret=secretKey)
        
    return template('admin_movies', title='Admin movies', res=listOut) 

@route('/add_movie')
@route('/add_movie/')
@route('/add_movie', method='GET')
def do_upload():
    
    loginName = CheckIfCorrectCredentials()

    if (loginName == "arttt1" or loginName == "szymm2"): 
    # Connect and insert new values for movie
        conn = sqlite3.connect('cinema_AS_SS.db')
        c = conn.cursor()
    
        if request.GET.save:
        
            new_id = request.query.id_movie
            new_movie_name = request.query.movie_name
            new_id_room = request.query.id_room
            new_date = request.query.date
            new_time = request.query.time
        
            insert_new_movies = (
            """INSERT INTO Movie(id_movie, movie_name, id_room, date, time)  VALUES(?, ?, ?, ?, ?)"""
            )
        
            new_movies = (new_id, new_movie_name, new_id_room, new_date, new_time)
        
            c.execute(insert_new_movies, new_movies)
                
            conn.commit()
            conn.close()
            
            return template('movie_added', title = "Movie added!")
    
        return template('add_movie', title = "Add movie")
    else:
        return '<p>You are not an admin. Access forbidden. </p>'

@route('/sold_tickets')
@route('/sold_tickets/')
def sold_tickets():
    loginName = CheckIfCorrectCredentials()

    if (loginName == "arttt1" or loginName == "szymm2"):   
    
        conn = sqlite3.connect('cinema_AS_SS.db')        
        c = conn.cursor()
        c.execute("SELECT login, type, count(*), sum(price) FROM Tickets GROUP BY login, type")
        listOut=c.fetchall()
        conn.commit()        
        conn.close()

        return template('sold_tickets', title='Sold tickets', res=listOut)
    
@route('/users')    
@route('/users/', methods=["GET", "POST"])
def index():
    
    loginName = CheckIfCorrectCredentials()
    
    if (loginName == "arttt1" or loginName == "szymm2"):   

        conn = sqlite3.connect('cinema_AS_SS.db')        
        c = conn.cursor()
        c.execute("SELECT * FROM Customers ")
        listOut=c.fetchall()
        conn.commit()        
        conn.close()

        return template('admin_users', title='Admin user', res=listOut)

    return template('index', title='Home page')



@route('/delete/<login:path>')
@route('/delete/<login:path>/')
def delete(login):
    
    conn = sqlite3.connect('cinema_AS_SS.db')        
    c = conn.cursor()
    c.execute("DELETE FROM Customers WHERE login = ?", (login,))
    conn.commit()        
    conn.close()
    redirect('/users')
    return True
    

@route('/my_reservation')
@route('/my_reservation/')
def profile():
    
    loginName = CheckIfCorrectCredentials()   
    conn = sqlite3.connect('cinema_AS_SS.db')
    c = conn.cursor()
    c.execute("SELECT a.id_ticket, b.movie_name, a.type, a.price, a.room, a.row, a.seat_number, b.date, b.time \
              FROM Tickets as a\
              LEFT JOIN Movie as b\
              on a.id_movie = b.id_movie\
              WHERE a.login == ?", (loginName, ))
    listOut=c.fetchall()
    conn.commit()
    conn.close()
    
    #loginName = request.get_cookie("user", secret=secretKey)
    if (loginName == "arttt1" or loginName == "szymm2"):
        return template('admin_profile', title='Admin Profile', res=listOut)      
    
    return template('my_reservation', title='My reservation', res=listOut) 

@route('/my_profile')
@route('/my_profile/')
def profile():
    
    loginName = CheckIfCorrectCredentials()   
    conn = sqlite3.connect('cinema_AS_SS.db')
    c = conn.cursor()
    c.execute("SELECT * \
              FROM Customers\
              WHERE login == ?", (loginName, ))
    listOut=c.fetchall()
    conn.commit()
    conn.close()
    
    #loginName = request.get_cookie("user", secret=secretKey)    
    
    return template('my_profile', title='My profile', res=listOut)

@route('/my_profile_update/<login:path>')
@route('/my_profile_update/<login:path>/')
@route('/my_profile_update/<login:path>', method='GET')
def profile_update(login):

    if request.GET.save:
        
        new_password = request.query.password
        new_name = request.query.name
        new_surname = request.query.surname
        new_mobile = request.query.mobile
        new_email = request.query.email
        new_address = request.query.address
    
        conn = sqlite3.connect('cinema_AS_SS.db')
        c = conn.cursor()
        
        update_existing_profile = ("UPDATE Details SET password = ?, name = ?, surname = ?, mobile = ?, email = ?, address = ? WHERE movie_name = ?")
        
        update_profile = (new_password, new_name, new_surname, new_mobile, new_email, new_address, login)
            
        c.execute(update_existing_profile, update_profile)
                
        conn.commit()
        conn.close()
    
        return '<p>You have successfully changed your credentials.</p>'
    
    else:
    
        conn = sqlite3.connect('cinema_AS_SS.db')
        c = conn.cursor()
        c.execute("SELECT password, name, surname, mobile, email, address FROM Customers")
        cur_data = c.fetchone()
        conn.commit()
        conn.close()
    
        return template('update_profile', title = 'Update Profile', existing_login = login, res = cur_data)
    

@route('/movies/<film:path>')
def submit(film):
    
    conn = sqlite3.connect('cinema_AS_SS.db')
    c = conn.cursor()
    c.execute("SELECT * \
              FROM Details \
              WHERE movie_name == ?", (film, ))
    listOut=c.fetchall()
    conn.commit()
    conn.close()
    
    return template('movie_details', title = 'Movie Details', res=listOut)

# To do usuwania filmow jak starczy czasu

@route('/delete_movie/<movie_name:path>')
@route('/delete_movie/<movie_name:path>/')
def delete_movie(movie_name):
    
    conn = sqlite3.connect('cinema_AS_SS.db')        
    c = conn.cursor()
    c.execute("DELETE FROM Movie WHERE movie_name = ?", (movie_name,))
    conn.commit()        
    conn.close()
    redirect('/admin_movies')
    return True


@route('/admin_movies/<film:path>')
def submit(film):
    
    loginName = CheckIfCorrectCredentials() 
    if (loginName == "arttt1" or loginName == "szymm2"): 
    
        loginName = CheckIfCorrectCredentials()  
        conn = sqlite3.connect('cinema_AS_SS.db')
        c = conn.cursor()
        c.execute("SELECT * \
              FROM Details \
              WHERE movie_name == ?", (film, ))
        listOut=c.fetchall()
        conn.commit()
        conn.close()
    
        return template('admin_movie_details', title = 'Movie Details', res=listOut)

    

@route('/change/<film:path>', method='GET')
def change_movie_detail(film):

    loginName = CheckIfCorrectCredentials() 
    if (loginName == "arttt1" or loginName == "szymm2"): 
    
        if request.GET.save:
        
            new_movie_name = request.GET.get('movie_name')
            new_movie_type = request.GET.get('movie_type')
            new_main_actor = request.GET.get('main_actor')
            new_description = request.GET.get('description')
    
            conn = sqlite3.connect('cinema_AS_SS.db')
            c = conn.cursor()
            
            change_movie_details = ("UPDATE Details SET movie_name = ?, movie_type = ?, main_actor = ?, description = ? WHERE movie_name = ?")
           
        
            change_details = (new_movie_name, new_movie_type, new_main_actor, new_description, film)
        
            c.execute(change_movie_details, change_details)
            conn.commit()
            conn.close()
            
            redirect('/admin_movies')
        else:
            conn = sqlite3.connect('cinema_AS_SS.db')
            c = conn.cursor()
            c.execute("SELECT * FROM Details WHERE movie_name LIKE ?", (film,))
            cur_data = c.fetchone()
         
        return template('edit_movie_detail', title = "Movie Details edition", film = film)
        return True
        
    else:
        return '<p> You are not an admin. Access denied'

@route('/admin_panel')
@route('/admin_panel/')    
def admin_panel():    
    loginName = CheckIfCorrectCredentials() 
    if (loginName == "arttt1" or loginName == "szymm2"): 
    
        conn = sqlite3.connect('cinema_AS_SS.db')
        c = conn.cursor()
        c.execute("SELECT id_ticket, id_movie, room, row, seat_number, count(*) as Duplicates\
                    FROM Tickets\
                    GROUP BY id_movie, room, row, seat_number")
        listOut = c.fetchall()
        conn.commit()
        conn.close()

        return template('admin_panel', title = 'Tickets', res=listOut)
        
    else:
        return '<p> You are not an admin. Access denied. </p>'
    
@route('/change_ticket/<id_ticket:path>')   
@route('/change_ticket/<id_ticket:path>/')    
@route('/change_ticket/<id_ticket:path>', method='GET')
def change_tickets(id_ticket):

    loginName = CheckIfCorrectCredentials() 
    if (loginName == "arttt1" or loginName == "szymm2"): 
    
        if request.GET.save:
        
            new_movie_row = request.GET.get('row')
            new_movie_seat_number = request.GET.get('seat_number')
    
            conn = sqlite3.connect('cinema_AS_SS.db')
            c = conn.cursor()
            
            change_ticket_place = ("UPDATE Tickets SET row = ?, seat_number = ? WHERE id_ticket = ?")
           
            change_details = (new_movie_row, new_movie_seat_number, id_ticket)
        
            c.execute(change_ticket_place, change_details)
            conn.commit()
            conn.close()
            
            redirect('/admin_panel')
         
        return template('edit_ticket_detail', title = "Movie Details edition", id_ticket = id_ticket)
        return True
        
# @route('/admin_panel')
# @route('/admin_panel/')
# def admin_panel():

#     if request.GET.save:
        
#         new_password = request.query.password
#         new_name = request.query.name
#         new_surname = request.query.surname
#         new_mobile = request.query.mobile
#         new_email = request.query.email
#         new_address = request.query.address
    
#         conn = sqlite3.connect('cinema_AS_SS.db')
#         c = conn.cursor()
        
#         update_existing_profile = ("UPDATE Details SET password = ?, name = ?, surname = ?, mobile = ?, email = ?, address = ? WHERE movie_name = ?")
        
#         update_profile = (new_password, new_name, new_surname, new_mobile, new_email, new_address, login)
            
#         c.execute(update_existing_profile, update_profile)
                
#         conn.commit()
#         conn.close()
    
#         return '<p>You have successfully changed your credentials.</p>'    
    
@route('/rooms')
@route('/rooms/')
def rooms():
    
    loginName = CheckIfCorrectCredentials() 
    if (loginName == "arttt1" or loginName == "szymm2"): 
    
        loginName = CheckIfCorrectCredentials()  
        conn = sqlite3.connect('cinema_AS_SS.db')
        c = conn.cursor()
        c.execute("SELECT * \
              FROM Rooms")
        listOut=c.fetchall()
        conn.commit()
        conn.close()
    
        return template('rooms', title = 'Cinema rooms', res=listOut)    
    

@route('/signout')
@route('/signout/')
@route('/signout', method="POST")
def signout():
    
    if (request.get_cookie("user", secret=secretKey) == None):
        redirect('/index')
        
    response.delete_cookie("user", path='/', domain='localhost')
    response.delete_cookie("password", path='/', domain='localhost')
    response.delete_cookie("randStr", path='/', domain='localhost')
    
    redirect('/signout_success')

    
@route('/signout_success')
@route('/signout_success/')
def signoutsuccess():
    return template('signout_success', title= 'Sign out')


# stare, moze sie przyda
# @route('/basic_table')
# @route('/basic_table/')
# def table():
#     merg = pd.read_csv("merged15.csv")
#     z = HTML(merg.to_html())
#     tables=merg.to_html()
#     #tables = HTML( merg.to_html().replace("\\n","<br>"))
#     return template('table', tabela=tables, zawartosc= '<strong><u>Tabela</u></strong><p></p>')
      
run(host='localhost', port=8001, debug=True)

Bottle v0.12.19 server starting up (using WSGIRefServer())...
Listening on http://localhost:8001/
Hit Ctrl-C to quit.

127.0.0.1 - - [29/Jan/2022 14:00:18] "GET / HTTP/1.1" 200 9276
127.0.0.1 - - [29/Jan/2022 14:00:18] "GET /static/css/bootstrap.min.css HTTP/1.1" 200 97339
127.0.0.1 - - [29/Jan/2022 14:00:19] "GET /static/css/bootstrap-theme.css HTTP/1.1" 200 118749
127.0.0.1 - - [29/Jan/2022 14:00:19] "GET /static/css/elegant-icons-style.css HTTP/1.1" 200 25252
127.0.0.1 - - [29/Jan/2022 14:00:19] "GET /static/css/font-awesome.min.css HTTP/1.1" 200 20766
127.0.0.1 - - [29/Jan/2022 14:00:19] "GET /static/assets/jquery-easy-pie-chart/jquery.easy-pie-chart.css HTTP/1.1" 200 141
127.0.0.1 - - [29/Jan/2022 14:00:19] "GET /static/css/owl.carousel.css HTTP/1.1" 200 1386
127.0.0.1 - - [29/Jan/2022 14:00:19] "GET /static/css/jquery-jvectormap-1.2.2.css HTTP/1.1" 200 781
127.0.0.1 - - [29/Jan/2022 14:00:19] "GET /static/css/widgets.css HTTP/1.1" 200 7203
127.0.0.1 - - [29/Jan/2022 14:00:19] "GE

False False

Present all data in student table:
[]


127.0.0.1 - - [29/Jan/2022 14:01:50] "GET /login/ HTTP/1.1" 200 8381
127.0.0.1 - - [29/Jan/2022 14:02:31] "GET /register? HTTP/1.1" 200 8955


False False

Present all data in student table:
[]


127.0.0.1 - - [29/Jan/2022 14:02:38] "GET /login HTTP/1.1" 200 8381


...

127.0.0.1 - - [29/Jan/2022 14:04:30] "GET /login_suc HTTP/1.1" 200 8319


szymm2 python2

Present all data in student table:
[]


127.0.0.1 - - [29/Jan/2022 14:04:44] "GET /users/ HTTP/1.1" 200 10201


szymm2 python2

Present all data in student table:
[]


127.0.0.1 - - [29/Jan/2022 14:04:57] "GET /sold_tickets/ HTTP/1.1" 200 9195
